In [26]:
import pandas as pd
import datetime
import json
import boto3
from datetime import date


# Dynamo db client
boto_client = boto3.resource("dynamodb")
dynamo_db_table = boto_client.Table("interconnector-data")

In [37]:
def hour_rounder(t):
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))

In [50]:
def get_date_keys():
    """Gets the date range of data 2 minutes before current"""
    date_keys = []
    end_range = hour_rounder(datetime.datetime.today())
    start_range = end_range - datetime.timedelta(hours=12)
    while start_range <= datetime.datetime.today():
        start_range += datetime.timedelta(minutes = 15)
        date_keys.append(int(start_range.strftime("%Y%m%d%H%M%S")))

    return date_keys

In [52]:
def get_todays_data_from_db():
    """
    Returns the data for today from the database
    """
    todays_date = int(datetime.datetime.today().strftime("%Y%m%d"))
    data = boto_client.batch_get_item(RequestItems = {"interconnector-data": {'Keys': [{'datetime': dt, 'date': todays_date} for dt in get_date_keys()]}})
        
    return data

In [53]:
get_todays_data_from_db()

{'Responses': {'interconnector-data': [{'France': Decimal('1876'),
    'datetime': Decimal('20221014144500'),
    'date': Decimal('20221014'),
    'Belgium': Decimal('705'),
    'Netherlands': Decimal('345'),
    'Norway': Decimal('-6')},
   {'France': Decimal('2742'),
    'datetime': Decimal('20221014174500'),
    'date': Decimal('20221014'),
    'Belgium': Decimal('125'),
    'Netherlands': Decimal('-425'),
    'Norway': Decimal('-1233')},
   {'France': Decimal('1783'),
    'datetime': Decimal('20221014124500'),
    'date': Decimal('20221014'),
    'Belgium': Decimal('604'),
    'Netherlands': Decimal('514'),
    'Norway': Decimal('108')},
   {'France': Decimal('1783'),
    'datetime': Decimal('20221014120000'),
    'date': Decimal('20221014'),
    'Belgium': Decimal('604'),
    'Netherlands': Decimal('449'),
    'Norway': Decimal('108')},
   {'France': Decimal('1737'),
    'datetime': Decimal('20221014133000'),
    'date': Decimal('20221014'),
    'Belgium': Decimal('749'),
    'Net

In [ ]:

def lambda_handler(event, context):
    
    return {
        'statusCode': 200,
        'body': get_todays_data_from_db()
    }
    